# Run pyCIAM in a Diaz 2016 configuration, using Diaz 2016 Inputs

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import distributed as dd
import pandas as pd
import xarray as xr
from pyCIAM.constants import CASE_DICT, SOLVCASES
from pyCIAM.io import load_diaz_inputs
from pyCIAM.run import calc_costs
from shared import (
    AUTHOR,
    CONTACT,
    HISTORY,
    PATH_DIAZ_INPUTS_INT,
    PATH_DIAZ_RES,
    PATH_PARAMS_DIAZ,
)

In [3]:
# number of seg to run at once
SEG_CHUNKSIZE = 4050

# parameter object
PARAMS = pd.read_json(PATH_PARAMS_DIAZ)["values"]

# attrs
DESCRIPTION = "Projected coastal damages from the Coastal Impact and Adaptation Model (CIAM, Diaz 2016), using original Diaz 2016 inputs and assumptions."
HISTORY = (
    """Replication of Diaz 2016.

"""
    + HISTORY
)

In [4]:
client = dd.Client(threads_per_worker=1, n_workers=3)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/bolliger32/proxy/8787/status,
Dashboard: /user/bolliger32/proxy/8787/status,Workers: 3
Total threads: 3,Total memory: 22.50 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36069,Workers: 3
Dashboard: /user/bolliger32/proxy/8787/status,Total threads: 3
Started: Just now,Total memory: 22.50 GiB
Comm: tcp://127.0.0.1:36015,Total threads: 1
Dashboard: /user/bolliger32/proxy/34811/status,Memory: 7.50 GiB
Nanny: tcp://127.0.0.1:44533,


## Define wrapper function

In [5]:
def diaz2016_wrapper(segs, diaz_flags=True):

    inputs, lsl = load_diaz_inputs(PATH_DIAZ_INPUTS_INT, segs, PARAMS)

    ncc_inputs = lsl.rcp_pt.str.startswith("rcp0")
    lsl_ncc = lsl.isel(rcp_pt=ncc_inputs)
    lsl_wcc = lsl.isel(rcp_pt=~ncc_inputs)

    kwargs = dict(
        dmf_kwargs={"floodmortality": PARAMS.floodmortality},
        elev_chunksize=None,
        diaz_protect_height=diaz_flags,
        diaz_construction_freq=diaz_flags,
        diaz_lslr_plan=diaz_flags,
        diaz_negative_retreat=diaz_flags,
        diaz_forward_diff=diaz_flags,
        diaz_zero_costs_in_first_year=diaz_flags,
        diaz_fixed_vars_for_onetime_cost=diaz_flags,
        diaz_calc_noadapt_damage_w_lslr=diaz_flags,
        diaz_storm_calcs=True,
        surge_lookup=None,
    )

    # calculate ciam costs for no SLR
    costs_ncc, refA = calc_costs(
        inputs, lsl_ncc, min_R_noadapt=None, return_year0_hts=True, **kwargs
    )
    costs_ncc = select_optimal_case(costs_ncc, inputs.dfact)

    # calculate optimal approach w/ no SLR
    refA = refA.sel(case=SOLVCASES).isel(case=costs_ncc.optimal_case, drop=True)

    costs_ncc = calc_costs(
        inputs, lsl_ncc, min_R_noadapt=refA, return_year0_hts=False, **kwargs
    )
    costs_ncc = select_optimal_case(costs_ncc, inputs.dfact)

    # reindex to load for each w/ SLR rcp
    refA["rcp_pt"] = (
        refA.rcp_pt.str.split(dim="tmp", sep="_").isel(tmp=1, drop=True).values
    )
    refA = refA.sel(
        rcp_pt=lsl_wcc.rcp_pt.str.split(dim="tmp", sep="_").isel(tmp=1, drop=True)
    )
    refA["rcp_pt"] = lsl_wcc.rcp_pt.values

    costs_wcc = calc_costs(
        inputs, lsl_wcc, min_R_noadapt=refA, return_year0_hts=False, **kwargs
    )
    costs_wcc = select_optimal_case(costs_wcc, inputs.dfact)

    return xr.concat((costs_ncc, costs_wcc), dim="rcp_pt")


def select_optimal_case(costs, dfact, npv_start=2010):
    this_costs = costs.sel(case=SOLVCASES)
    optimal_case = (
        (this_costs.sum("costtype") * dfact)
        .sel(year=slice(npv_start, None))
        .sum("year")
        .argmin("case")
        .astype("uint8")
    )
    costs = xr.concat(
        (
            this_costs,
            this_costs.isel(case=optimal_case, drop=True).expand_dims(
                case=["optimalfixed"]
            ),
        ),
        dim="case",
    )
    return xr.Dataset({"costs": costs, "optimal_case": optimal_case})

## Run model

In [6]:
segs = xr.open_zarr(PATH_DIAZ_INPUTS_INT).seg.values
seg_grps = [segs[ix : ix + SEG_CHUNKSIZE] for ix in range(0, len(segs), SEG_CHUNKSIZE)]

In [7]:
out = xr.concat(
    client.gather(client.map(diaz2016_wrapper, seg_grps)),
    dim="seg",
)

## Save

In [11]:
out.attrs.update(
    {
        "author": AUTHOR,
        "history": HISTORY,
        "contact": CONTACT,
        "description": DESCRIPTION,
        "optimal_case_dict": str(CASE_DICT),
    }
)

In [12]:
for k, v in out.coords.items():
    if v.dtype == object:
        out[k] = v.astype("unicode")
out.to_zarr(PATH_DIAZ_RES, mode="w")